# Exploring the system of many particles with closed boundaries

Here we do some tests of the system for different parameter sets. We were able to reproduce the synchronous rotation phase, the asynchronous rotation phase, and the rupture phase, but not the exchange phase. The exchange could require periodic boundaries. 

In [9]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import trackpy as tp
import string as st


from IPython.display import HTML

%matplotlib inline
%reload_ext autoreload
%autoreload 2

# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

# Default Parameters

This is the base function we use to run the simulation. We define a given (approximate) set of particles and then we adjust the area of the region to obtain a given packing fraction

In [4]:
def run_simulation(height, packing, angle=27, frequency = 1, radius = 1.4, particles = 1200):
    
    initial_sep = 3
    part_edge = np.round(np.sqrt(particles))*initial_sep
    x_loc = np.arange(-part_edge/2,part_edge/2+1,initial_sep)
    y_loc = np.arange(-part_edge/2,part_edge/2+1,initial_sep)

    [X,Y] = np.meshgrid(x_loc,y_loc)
    Z = np.zeros(np.shape(X))

    initial_positions = np.array([[x,y,z] for (x,y,z) in zip(X.flatten(),Y.flatten(),Z.flatten())])
    
    area_particle = len(initial_positions)*radius**2*np.pi
    area_region = area_particle/packing

    length_region = np.sqrt(area_region)
    length_particles = len(x_loc)*initial_sep

    if length_region<length_particles:
        raise ValueError("packing is too high")

    Region = [np.round(length_region),np.round(length_region),height]
    
    sim = lmp.sim(initial_positions,
              magnitude = 7.28,
              file_name = "confined_system_explore",
              dir_name = "ExploreNb",
              radius = radius,
              diffusion = 0.14,
              frequency=frequency,
              susceptibility = 0.4,
              angle=angle,
              framerate=1e1,
              timestep=1e-4,
              total_time = 20,
              temperature = 300,
              stamp_time = True,
              space = {'region':Region,'boundary':['f','f','f'],'walls':[True,True,True]})
    sim.generate_scripts()
    
    sim.run()
    return sim

def display(sim):
    trj = sim.load(read_trj=True)
    anim = animate_trj(trj,sim)
    anim.save(sim.base_name+".gif",writer = "imagemagick")
    video_html = st.Template(""" <video controls>
          <source src="$name" type="video/mp4">
            </video> """).substitute(name=sim.base_name+".mp4")
    HTML(video_html)
    return anim

def display(sim):
    trj = sim.load(read_trj=True)
    anim = animate_trj(trj,sim)
    anim.save(sim.base_name+".gif",writer = "imagemagick")
    video_html = st.Template(""" <video controls>
          <source src="$name" type="video/mp4">
            </video> """).substitute(name=sim.base_name+".mp4")
    HTML(video_html)
    return anim

def direct_display(sim):
    trj = sim.load(read_trj=True)
    anim = animate_trj(trj,sim)
    return anim.to_html5_video()

## High packing, low frequency. Synchronous rotation

In [34]:
sim = run_simulation(height = 4, packing = 0.5, frequency = 1,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
HTML(video_html)

## Low Packing, medium frequency. Synchronous Rotation

In [36]:
sim = run_simulation(height = 4.2, packing = 0.3, frequency = 3,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)


      <img src="ExploreNb/confined_system_explore_2018_02_18_19_25_37.gif" type="video/gif">


## Low Packing, high frequency. Rupture.

However, I expected asynchronous rotation. But I shouldnt because of the confinement

In [37]:
sim = run_simulation(height = 4.2, packing = 0.3, frequency = 10,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)


      <img src="ExploreNb/confined_system_explore_2018_02_18_19_32_38.gif" type="video/gif">


## Low confinement. Low packing, high frequency. Unstable simulation

In [8]:
sim = run_simulation(height = 5, packing = 0.3, frequency = 10,particles = 500)
anim = display(sim)
video_html = st.Template("""
     <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)

KeyError: (113000.0, 1.0)

## Very high packing, medium frequency. Synchronous/rupture. No exchange

In [41]:
sim = run_simulation(height = 4.2, packing = 0.55, frequency = 3,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)


      <img src="ExploreNb/confined_system_explore_2018_02_18_19_57_25.gif" type="video/gif">


## Very high packing, medium high frequency. Rupture. No exchange.

In [42]:
sim = run_simulation(height = 4.2, packing = 0.55, frequency = 6,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)


      <img src="ExploreNb/confined_system_explore_2018_02_18_20_07_12.gif" type="video/gif">


## High confinement. High packing, medium frequency. Rupture

In [44]:
sim = run_simulation(height = 3.5, packing = 0.5, frequency = 6,particles = 500)
anim = display(sim)
video_html = st.Template("""
      <img src="$name" type="video/gif">""").substitute(name=sim.base_name+".gif")
print(video_html)
HTML(video_html)


      <img src="ExploreNb/confined_system_explore_2018_02_18_20_26_41.gif" type="video/gif">


# Support Functions

In [6]:
def animate_trj(trj,sim,ax=False):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    idx = pd.IndexSlice
    
    if not ax:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    particles = trj.index.get_level_values('id').unique()
    n_of_particles = len(trj.index.get_level_values('id').unique())
    
    region = sim.sim_parameters.space["region"] 
    radius = sim.particle_properties[0].radius
    
    framerate = sim.run_parameters.framerate
    runtime = sim.run_parameters.total_time
    timestep = sim.run_parameters.timestep
    
    lammps_time = 1e6;
    
    dt_data = timestep*framerate*lammps_time # Data timestep in lammps_time
    dt_video = 1/framerate*1000 # video timestep in miliseconds
    frames = runtime*framerate+1
    
    ax.set_xlim(region[0],region[1])
    ax.set_ylim(region[2],region[3])
    ax.set(aspect='equal')
    ax.set_xlabel("$x [\mu{m}]$")
    ax.set_ylabel("$y [\mu{m}]$")
    
    patches = []
    for i,p in enumerate(particles):
        c = plt.Circle((0, 0), radius)
        patches.append(c)

    p = clt.PatchCollection(patches, cmap=plt.cm.RdBu)
    p.set_array(np.zeros(0))
    p.set_clim([region[4]+radius,region[5]-radius])
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0)
    plt.colorbar(p,label='$z [\mu{m}]$',cax=cax)

    def init():
        ax.add_collection(p)
        return p,

    def animate(frame):

        for (part_id,particle) in enumerate(particles):
            patches[part_id].center = (trj.loc[idx[frame*dt_data,particle],'x'],trj.loc[idx[frame*dt_data,particle],'y'])
        p.set_paths(patches)
        p.set_array(trj.loc[idx[frame*dt_data,:],'z'].values)
        ax.add_collection(p)
        return p,

    anim = anm.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(frames), interval=dt_video, blit=True);
    plt.close(anim._fig)

    return anim